In [1]:
from pyVHR.datasets.ubfc2 import UBFC2
from pyVHR.datasets.dataset import Dataset
from pyVHR.datasets.dataset import datasetFactory
from pyVHR.methods.base import methodFactory
from pyVHR.signals.video import Video
from copy import copy
import numpy as np
from tensorflow.python.keras.utils import np_utils

In [2]:
LENGTH_VIDEO = 60 
IMAGE_WIDTH = 25 
IMAGE_HEIGHT = 25 
IMAGE_CHANNELS = 1 
RATE = 30
NB_SECOND = int(LENGTH_VIDEO / RATE)
# Available Outputs
HEART_RATES = np.linspace(55, 240, 75)
NB_CLASSES = len(HEART_RATES)


def extractDataFromVideo(videoFilename, GTFilename):
    
    sigGT = dataset.readSigfile(GTFilename)
    winSizeGT = NB_SECOND
    bpmGT, timesGT = sigGT.getBPM(winSizeGT)
    
    bpm = np.round(bpmGT)
    bpm = bpm - 55
    bpm = np.round(bpm / 2.5)
    
    video = Video(videoFilename)
    video.getCroppedFaces(detector='dlib', extractor='skvideo')
    video.setMask(typeROI='skin_adapt',skinThresh_adapt=0.22)

    NB_LAPSE = int(video.numFrames / RATE)

    imgs = np.zeros(shape=(video.numFrames, video.cropSize[0], video.cropSize[1], 1))
    xtest = np.zeros(shape=(NB_LAPSE, LENGTH_VIDEO, IMAGE_HEIGHT , IMAGE_WIDTH, 1))
    ytest = np.zeros(shape=(NB_LAPSE, NB_CLASSES + 1))

    # prepare labels and label categories
    labels = np.zeros(NB_CLASSES + 1)

    for i in range(NB_CLASSES + 1):
        labels[i] = i
    labels_cat = np_utils.to_categorical(labels)
 
    # channel extraction
    if (video.cropSize[2]<3):
        IMAGE_CHANNELS = 1
    else:
        IMAGE_CHANNELS = video.cropSize[2]

    # load images (imgs contains the whole video)
    for j in range(video.numFrames):

        if (IMAGE_CHANNELS==3):
            temp = video.faces[j]/255
            temp = temp[:,:,1]      # only the G component is currently used
        else:
            temp = video.faces[j] / 255

        imgs[j] = np.expand_dims(temp, 2)
    


    for lapse in range(0,NB_LAPSE):  
        xtemp = np.zeros(shape=(LENGTH_VIDEO, IMAGE_HEIGHT , IMAGE_WIDTH, 1)) 
    
        start = lapse * RATE
        end = start + LENGTH_VIDEO
        if(end > video.numFrames):
            break
        c=0
    
        for j in range(start,end-1):    
            faceCopy = copy(imgs[j])
            #randomization pixel choice 
            np.random.shuffle(faceCopy)
            for m in range(0, IMAGE_HEIGHT):
                for n in range(0, IMAGE_WIDTH):
                    xtemp[c][m][n]= faceCopy[m][n]
            c = c +1
                
        xtest[lapse] = np.expand_dims(xtemp, 0)
        ytest[lapse] = np.expand_dims(labels_cat[int(bpm[lapse+NB_SECOND])],0)
    return xtest, ytest

In [3]:
dataset = datasetFactory("UBFC2")

xtrain = np.array(np.zeros(shape=(0,LENGTH_VIDEO, IMAGE_HEIGHT, IMAGE_WIDTH, 1)))
ytrain = np.zeros(shape=(0, NB_CLASSES + 1))

for i in range (len(dataset.videoFilenames)):
    xtest, ytest = extractDataFromVideo(dataset.videoFilenames[i], dataset.sigFilenames[i])
    xtrain = np.concatenate((xtrain, xtest), axis=0)
    ytrain = np.concatenate((ytrain, ytest), axis=0)

indices = np.arange(xtrain.shape[0])
np.random.shuffle(indices)

xtrain = xtrain[indices]
ytrain = ytrain[indices]

np.savez('./data.npz', a=xtrain, b=ytrain)
print(np.shape(xtrain))

c:\users\florian\appdata\local\programs\python\python36\lib\site-packages\pyVHR\signals\video.py:71: UserWarning:


WARNING!! Requested detector method is different from the saved one




(1629, 60, 25, 25, 1)


In [4]:
data = np.load('./data.npz')

pct = 0.9
sizeDataset = data['a'].shape[0]
sizeTrainData = int(sizeDataset * pct) 

xtrain = data['a'][:sizeTrainData,:]
xvalidation = data['a'][sizeTrainData:,:]

ytrain = data['b'][:sizeTrainData,:]
yvalidation = data['b'][sizeTrainData:,:]

np.savez('./dataSplited.npz', a=xtrain, b=ytrain, c=xvalidation, d=yvalidation)

print(np.shape(xtrain))
print(np.shape(ytrain))
print(np.shape(xvalidation))
print(np.shape(yvalidation))


(1466, 60, 25, 25, 1)
(1466, 76)
(163, 60, 25, 25, 1)
(163, 76)
